In [34]:
import warnings
import os
import pandas as pd
import numpy as np
import tqdm

import sys

sys.path.insert(1, '/Users/gosha/Desktop/DS Competitions/MTS Cup/src')

warnings.filterwarnings('ignore')

In [2]:
# импортируем свои функции для обработки данных
from src.preprocessing import get_data_part_day, get_data_days, 
    get_user_model_price, get_user_url_cnt, get_user_city_cnt

# Описание задачи

## О соревновании

https://ods.ai/competitions/mtsmlcup

Задача соревнования
- Определение пола и возраста владельца HTTP cookie по истории активности пользователя в интернете на основе синтетических данных.

Метрики соревнования:
* ROC-AUC – для определения пола, f1 weighted – для определения возраста.
* Все решения рассчитываются по формуле -  2 * f1_weighted(по 6 возрастным бакетам) + gini по полу.
* Возрастные бакеты (Класс 1 — 19-25, Класс 2 — 26-35, Класс 3 — 36-45, Класс 4 — 46-55, Класс 5 — 56-65, Класс 6 — 66+).

Описание колонок файла с данными:
* 'region_name' – Регион
* 'city_name' – Населенный пункт
* 'cpe_manufacturer_name' – Производитель устройства
* 'cpe_model_name' – Модель устройства
* 'url_host' – Домен, с которого пришел рекламный запрос
* 'cpe_type_cd' – Тип устройства (смартфон или что-то другое)
* 'Cpe_model_os_type' – Операционка на устройстве
* 'price' – Оценка цены устройства
* 'date' – Дата
* 'part_of_day' – Время дня (утро, вечер, и тд)
* 'request_cnt' – Число запросов одного пользователя за время дня (поле part_of_day)
* 'user_id' – ID пользователя

Описание колонок файла с таргетами:

* 'age' – Возраст пользователя
* 'is_male' – Признак пользователя : мужчина (1-Да, 0-Нет)
* 'user_id' – ID пользователя

## Структура проекта

Данная работа была разделена на несколько jupyter ноутбуков:

0. Data_preparing.ipnb - аггрегация отдельных файлов по user_id и склейка в финальный датасет
1. EDA.ipynb - исследовательская часть
2. baseline.ipynb - бейзлайн модели
3. create_embeddings.ipynb - создание эмбеддингов для дальнейшего их использования в качестве фич
4. baseline_embeddings.ipynb - бейзлан модели с эмбеддингами
5. model_tuning.ipynb - тюнинг наиболее перспективных моделей
6. gender_prediction_stacking.ipynb - стекинг моделей для предсказания пола

# Data preparing

В данном jupyter-ноутбуке мы рассмотрим и предобработаем 1 из 10 файлов с данными. После чего аггрегируем отдельно каждый файл в цикле и соединим в 1 датафрейм

In [3]:
df = pd.read_parquet(
    '../competition_data_final_pqt/part-00000-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet',
    engine='fastparquet')

In [4]:
df.shape

(32638709, 12)

In [21]:
df.head(3)

,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,date,part_of_day,request_cnt,user_id
0,Краснодарский край,Краснодар,Apple,iPhone 7,ad.adriver.ru,smartphone,iOS,20368.0,2022-06-15,morning,1,45098
1,Краснодарский край,Краснодар,Apple,iPhone 7,apple.com,smartphone,iOS,20368.0,2022-06-19,morning,1,45098
2,Краснодарский край,Краснодар,Apple,iPhone 7,avatars.mds.yandex.net,smartphone,iOS,20368.0,2022-06-12,day,1,45098


In [5]:
# поменяем тип числовых признаков, чтобы сэкономить память
df = df.astype({'request_cnt': 'int8', 'price': 'float32', 'user_id': 'int32'})

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32638709 entries, 0 to 32638708
Data columns (total 12 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   region_name            object        
 1   city_name              object        
 2   cpe_manufacturer_name  object        
 3   cpe_model_name         object        
 4   url_host               object        
 5   cpe_type_cd            object        
 6   cpe_model_os_type      object        
 7   price                  float32       
 8   date                   datetime64[ns]
 9   part_of_day            object        
 10  request_cnt            int8          
 11  user_id                int32         
dtypes: datetime64[ns](1), float32(1), int32(1), int8(1), object(8)
memory usage: 2.5+ GB


In [24]:
df.isna().sum()

region_name                   0
city_name                     0
cpe_manufacturer_name         0
cpe_model_name                0
url_host                      0
cpe_type_cd                   0
cpe_model_os_type             0
price                    674264
date                          0
part_of_day                   0
request_cnt                   0
user_id                       0
dtype: int64

In [6]:
def replace_model_mistakes(data: pd.DataFrame) -> pd.DataFrame:
    """
    Функция исправляет неточности в данных, заменяет
    :param data: датафрейм с данными
    :return: датафрейм с исправленными неточностями
    """
    # заменим значения, которые являются одинаковыми, но записаны по-разному
    data.cpe_model_os_type.replace('Apple iOS', 'iOS', inplace=True)
    data.cpe_manufacturer_name.replace('Huawei Device Company Limited',
                                       'Huawei',
                                       inplace=True)
    data.cpe_manufacturer_name.replace(
        'Motorola Mobility LLC, a Lenovo Company', 'Motorola', inplace=True)
    data.cpe_manufacturer_name.replace('Sony Mobile Communications Inc.',
                                       'Sony',
                                       inplace=True)

    # в таблицах для Nokia 3 Dual попадаются разные типы устройства, оставим только один тип
    data['cpe_type_cd'] = np.where((data['cpe_manufacturer_name'] == 'Nokia') &
                                   (data['cpe_model_name'] == '3 Dual'),
                                   'plain', data['cpe_type_cd'])

    # заполним пропуски цены несуществующим значением
    data.price.fillna(-999, inplace=True)

    return data

In [7]:
df = replace_model_mistakes(df)

# Feature Generation

## Количество запросов в разные части дня

In [10]:
"""
Аггрегируем датафрейм по user_id и генерируем новые признаки. Итоговый датафрейм содержит признаки:
- part_of_day_day - кол-во визитов в часть дня day
- part_of_day_evening - кол-во визитов в часть дня evening
- part_of_day_morning - кол-во визитов в часть дня morning
- part_of_day_night - кол-во визитов в часть дня night
- sum_visits - общее кол-во визитов
- day_pct - доля визитов в часть дня day
- evening_pct - доля визитов в часть дня evening
- morning_pct - доля визитов в часть дня morning
- night_pct - доля визитов в часть дня night
"""
df_part_day = get_data_part_day(df)
df_part_day.head(3)

,user_id,part_of_day_day,part_of_day_evening,part_of_day_morning,part_of_day_night,sum_visits,day_pct,evening_pct,morning_pct,night_pct
0,4,199,170,212,10,591,0.336717,0.287648,0.358714,0.016920
1,16,443,321,330,137,1231,0.359870,0.260764,0.268075,0.111292
2,18,566,114,411,34,1125,0.503111,0.101333,0.365333,0.030222


## Количество дней активности, среднее кол-во запросов в сутки

In [11]:
"""
Аггрегируем датафрейм по user_id и генерируем новые признаки. Итоговый датафрейм содержит признаки:
- act_days - кол-во дат активности пользователя
- request_cnt - кол-во запросов пользователя
- avg_req_per_day - среднее кол-во запросов пользователя
- period_days - кол-во дней между первым и последним визитом пользователя
- request_std - стандартное отклонение по количеству запросов
- act_days_pct - доля дней, когда пользователь совершал визит
"""
df_days = get_data_days(df)
df_days.head(3)

,user_id,act_days,request_cnt,avg_req_per_day,period_days,requests_std,act_days_pct
0,4,20,777,38.850000,43,0.647632,0.465116
1,16,64,2529,39.515625,74,1.397836,0.864865
2,18,32,1950,60.937500,38,1.204972,0.842105


## Цена устройства

In [12]:
"""
Аггрегируем датафрейм по user_id и генерируем новые признаки. Итоговый датафрейм содержит признаки:
- cpe_type_cd - тип устройства
- cpe_model_os_type - операционная система устройства
- cpe_manufacturer_name - производитель устройства
- price - цена устройства пользователя
"""
df_model = get_user_model_price(df)
df_model.head(3)

,user_id,cpe_type_cd,cpe_model_os_type,cpe_manufacturer_name,price
0,4,smartphone,Android,Huawei,12990.0
1,16,smartphone,Android,Samsung,9583.0
2,18,smartphone,Android,Samsung,22887.0


## Регион и город

In [13]:
"""
Аггрегируем датафрейм по user_id и генерируем новые признаки. Итоговый датафрейм содержит признаки:
- region_cnt - кол-во уникальных регионов, из которых был совершен визит
- city_cnt - кол-во уникальных городов, из которых был совершен визит
"""
df_city_cnt = get_user_city_cnt(df)
df_city_cnt.head(3)

,user_id,region_cnt,city_cnt
0,4,5,9
1,16,1,1
2,18,1,2


## URLs

In [14]:
"""
Аггрегируем датафрейм по user_id и генерируем новые признаки. Итоговый датафрейм содержит признаки:
- url_host_cnt - кол-во уникальных регионов, из которых был совершен визит
"""
df_url = get_user_url_cnt(df)
df_url.head(3)

,user_id,url_host_cnt
0,4,108
1,16,50
2,18,141


# Итоговый датафрейм

Теперь проделаем те же действия для каждого файла с данными и объединим все в общий датафрейм

In [28]:
# название папки с файлами
DATA_FOLD = '../competition_data_final_pqt'

df_final = pd.DataFrame()
for file in tqdm.tqdm_notebook(os.listdir(DATA_FOLD)):
    if file.endswith('.parquet'):
        temp_df = pd.read_parquet(f'{DATA_FOLD}/{file}', engine='fastparquet')
        temp_df = replace_model_mistakes(temp_df)

        # получаем аггрегированные данные
        data_part_day = get_data_part_day(temp_df)
        data_days = get_data_days(temp_df)
        data_user_model = get_user_model_price(temp_df)
        data_city_cnt = get_user_city_cnt(temp_df)
        data_url_cnt = get_user_url_cnt(temp_df)

        # объединяем все в один датафрейм
        temp_df_agg = (data_part_day
                       .merge(data_days, how='left', on='user_id')
                       .merge(data_user_model, how='left', on='user_id')
                       .merge(data_city_cnt, how='left', on='user_id')
                       .merge(data_url_cnt, how='left', on='user_id'))

        # добавляем аггрегированные данные в итоговый датафрейм
        df_final = pd.concat([df_final, temp_df_agg])

  0%|          | 0/11 [00:00<?, ?it/s]

In [29]:
df_final.head()

,user_id,part_of_day_day,part_of_day_evening,part_of_day_morning,part_of_day_night,sum_visits,day_pct,evening_pct,morning_pct,night_pct,...,period_days,requests_std,act_days_pct,cpe_type_cd,cpe_model_os_type,cpe_manufacturer_name,price,region_cnt,city_cnt,url_host_cnt
0,13,467,447,411,226,1551,0.301096,0.288201,0.264990,0.145712,...,116,1.618662,0.974138,smartphone,Android,BQ Devices Limited,-999.0,1,1,83
1,41,7,13,9,9,38,0.184211,0.342105,0.236842,0.236842,...,4,1.562360,1.000000,smartphone,Android,Xiaomi,12343.0,1,3,15
2,69,457,346,314,30,1147,0.398431,0.301656,0.273758,0.026155,...,60,1.331956,0.933333,smartphone,iOS,Apple,16657.0,1,1,91
3,71,49,57,12,7,125,0.392000,0.456000,0.096000,0.056000,...,8,0.865094,1.000000,smartphone,iOS,Apple,38037.0,2,2,35
4,85,838,1067,602,333,2840,0.295070,0.375704,0.211972,0.117254,...,41,1.144101,0.926829,smartphone,iOS,Apple,55809.0,2,6,174


In [30]:
df_final.shape

(415317, 23)

In [32]:
# проверим, что файлы не пересекались по user_id
(df_final.groupby('user_id', as_index=False)
         .agg({'city_cnt': 'count'})
         .query('city_cnt > 1'))

,user_id,city_cnt


In [33]:
df_final.to_csv('../data_agg/df_final.csv', index=False)